# Fine-tune hawkears
This notebook uses Hawkears v0.1.0 loaded from the bioacoustic model zoo and Opensoundscape 0.11.0. Note that changes to the implementation may be required for other veresions of HawkEars. 

imports

In [1]:
from custom_lr_hawkears import CustomLRHawkEars
import pandas as pd

### load annotations

this code assumes you have multi-hot labels arranged in a csv where the first 3 columns are
file, start_time, end_time
and there is one subsequent column per class, with 0 for absent / 1 for present
see opensoundscape.org annotation tutorials for how to create labels in this format


In [ ]:
train_df = pd.read_csv("path/to/train_multihot_labels.csv", index_col=[0, 1, 2])
val_df = pd.read_csv("path/to/validation_multihot_labels.csv", index_col=[0, 1, 2])

### initialize the custom model object
Initialize the model object using the CustomLRHawkEars class defined in the custom_lr_hawkears.py file. This class modifies the default Hawkears by setting the feature extractor's learning rate to be 1/10th of the classifier's learning rate. It provides an implementation of change_classes() method that works with Hawkears' internal model ensemble.

It will download the pre-trained checkpoints to the current working directory by default.

In [ ]:
hawkears = CustomLRHawkEars()
# change the output classes from the default set of species to the ones we are fine-tuning on
hawkears.change_classes(["SpecialBirdCall"])

Downloaded completed: hgnet1.ckpt
Loading model from local checkpoint /Users/SML161/bioacoustics-cookbook/hawkears_finetune/hgnet1.ckpt...


/Users/SML161/miniconda3/envs/opso0110/lib/python3.11/site-packages/bioacoustics_model_zoo/hawkears/hawkears.py:198: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mdict = to

Downloaded completed: hgnet2.ckpt
Loading model from local checkpoint /Users/SML161/bioacoustics-cookbook/hawkears_finetune/hgnet2.ckpt...
Downloaded completed: hgnet3.ckpt
Loading model from local checkpoint /Users/SML161/bioacoustics-cookbook/hawkears_finetune/hgnet3.ckpt...
Downloaded completed: hgnet4.ckpt
Loading model from local checkpoint /Users/SML161/bioacoustics-cookbook/hawkears_finetune/hgnet4.ckpt...
Downloaded completed: hgnet5.ckpt
Loading model from local checkpoint /Users/SML161/bioacoustics-cookbook/hawkears_finetune/hgnet5.ckpt...


/Users/SML161/miniconda3/envs/opso0110/lib/python3.11/site-packages/opensoundscape/ml/cnn.py:621: UserWarning: 
                    This architecture is not listed in opensoundscape.ml.cnn_architectures.ARCH_DICT.
                    It will not be available for loading after saving the model with .save() (unless using pickle=True). 
                    To make it re-loadable, define a function that generates the architecture from arguments: (n_classes, n_channels) 
                    then use opensoundscape.ml.cnn_architectures.register_architecture() to register the generating function.

                    The function can also set the returned object's .constructor_name to the registered string key in ARCH_DICT
                    to avoid this warning and ensure it is reloaded correctly by opensoundscape.ml.load_model().

                    See opensoundscape.ml.cnn_architectures module for examples of constructor functions
                    
  warnings.warn(
/Users/SML161/min

Train the custom architecture

In [ ]:
# train
hawkears.train(
    train_df=train_df,
    validation_df=val_df,
    epochs=10,
    batch_size=64,
    num_workers=8,
    save_interval=10,
    save_path="/path/to/save/dir",
)